In [1]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import matplotlib.pyplot as plt
import wget
import dask
import os
from tqdm import tqdm
import glob
import os

from numpy import linalg as LA
import math
import sklearn.preprocessing
import bahc
import pyRMT

%matplotlib inline

## Download k-line spot data by every hour from binance

In [2]:
url_template = r"https://data.binance.vision/data/spot/daily/klines/{0}/1h/{0}-1h-{1}.zip"
dir_template = r"data/spot/daily/klines/{0}/1h/{0}-1h-{1}.zip"


# @dask.delayed
def download_one_date(url, path):
    try:
        tmp = wget.download(url, out=path)
        return True
    except:
        # print("{} download failed".format(url))
        return False

def download_all_dates(token_pair: str, dates, path):
    first_meet = False
    for each in dates:
        res = download_one_date(url_template.format(token_pair, str(each.date())), path)
        if first_meet == False and res == True:
            first_meet = True
        if first_meet == True and res == False:
            break
    
def get_asset_pairs(x):
    with open("asset_pairs.txt", "r") as f:
        names = f.read()
    names = names.replace("\t", "")
    names = names.replace("\n", "")
    names = names.split("/")
    x_names = list(filter(lambda each: each.endswith(x), names))
    print("x: {} results length: {}".format(x, len(x_names)))
    return x_names

@dask.delayed
def main_download(pair, dates):
    path = "data/spot/daily/klines/{0}/1h".format(pair)
    if not os.path.exists(path):
        os.makedirs(path)
    download_all_dates(pair, dates, path)
    

In [3]:
USDT_pairs = get_asset_pairs("USDT")
dates = pd.date_range(start="2021-03-01",end="2023-01-15")
print(len(list(dates)))

x: USDT results length: 394
686


### Uncomment the following block to download all the raw data (it takes around 12 hours to download)

In [4]:
# promises = [main_download(each, dates) for each in USDT_pairs]
# dask.compute(promises)

## Data loading & pre-processing

In [5]:
@dask.delayed
def process_raw(pair, path):
    names = [
        "Open time",
        "Open",
        "High",
        "Low",
        "Close",
        "Volume",
        "Close time",
        "Quote asset volume",
        "Number of trades",
        "Taker buy base asset volume",
        "Taker buy quote asset volume",
        "Ignore",
    ]
    asset_data = pd.read_csv(path, names=names, header=None)
    asset_data["time"] = pd.to_datetime(asset_data["Open time"], unit='ms')
    asset_data[pair] = asset_data["Close"]
    date_indexed = asset_data.set_index("time")
    date_indexed.drop([
        "Open time",
        "Open",
        "High",
        "Low",
        "Close",
        "Volume",
        "Close time",
        "Quote asset volume",
        "Number of trades",
        "Taker buy base asset volume",
        "Taker buy quote asset volume",
        "Ignore",
        ], axis=1, inplace=True)

    return date_indexed


def load_one_pair(pair):
    files = glob.glob("data/spot/daily/klines/{}/1h/*".format(pair))
    files = [each for each in files if "(" not in each]
    if len(files) == 0:
        print(f"{pair} is empty, no files found")
        return False, None
    tasks = [process_raw(pair, each) for each in files]
    p_data_arr = dask.compute(tasks)
    result = pd.concat(p_data_arr[0])
    return True, result

def merge_assets(pd_arr, col: str):
    assets_close_matrix = pd_arr[0]
    for each in pd_arr[1:]:
        assets_close_matrix = assets_close_matrix.merge(each, how="outer", on=col)
        if assets_close_matrix.shape[0] > 16464:
            print(f"{each.columns} wrong rows: {assets_close_matrix.shape[0]}")
            assert True==False
    return assets_close_matrix

In [6]:
def main_load_and_merge_all_assets():
    existing_pairs = os.listdir("data/spot/daily/klines")
    print("Number of pairs: ", len(existing_pairs))
    chunk_sz = 10
    subsets = [existing_pairs[i:i + chunk_sz] for i in range(0, len(existing_pairs), chunk_sz)]
    # print(sum([len(each) for each in subsets]))
    for i in tqdm(range(len(subsets))):
        chunk = subsets[i]
        tmp_assets_arr = list()
        for each in chunk:
            success, tmp_asset = load_one_pair(each)
            if success:
                if tmp_asset.shape[0] > 16464:
                    print(f"{each} wrong rows: {tmp_asset.shape[0]}")
                tmp_assets_arr.append(tmp_asset)
        # tmp_assets_arr = [load_one_pair(each) for each in chunk]
        tmp_merge_result = merge_assets(tmp_assets_arr, "time")
        tmp_merge_result.to_pickle(f"data/clean/tmp_merge_{i}.pkl")
        
    
def inspect_assets_shape():
    existing_pairs = os.listdir("data/spot/daily/klines")
    print("Number of pairs: ", len(existing_pairs))
    # assets_pd_arr = [load_one_pair(each) for each in existing_pairs]
    for pair in existing_pairs:
        success, tmp_asset = load_one_pair(pair)
        assert tmp_asset.shape[0] <= 16464, f"{pair} wrong rows: {tmp_asset.shape[0]}"

# inspect_assets_shape()

### Uncomment the following block to merge single files into chunks (it takes long)

In [7]:
# res = main_load_and_merge_all_assets()

In [8]:
def merge_all_chunks(arr_id):
    df_arr = [pd.read_pickle(f"data/clean/tmp_merge_{i}.pkl") for i in arr_id]
    # for each in df_arr:
    #     print(each.shape)
    res = merge_assets(df_arr, "time")
    return res

### Uncomment the following block to merge all chunks (it takes long)

In [9]:
# res = merge_all_chunks([i for i in range(40)])
# res.to_pickle("data/clean/whole_usdt_merge.pkl")
# print(res.shape)

## MVP and covariance leaning functions

In [10]:
def eigenvalue_clipping(lambdas,v,lambda_plus):
    N=len(lambdas)
    
    # _s stands for _structure below
    sum_lambdas_gt_lambda_plus=np.sum(lambdas[lambdas>lambda_plus])
    
    sel_bulk=lambdas<=lambda_plus                     # these eigenvalues come from the seemingly random bulk
    N_bulk=np.sum(sel_bulk)
    sum_lambda_bulk=np.sum(lambdas[sel_bulk])        
    delta=sum_lambda_bulk/N_bulk                      # delta is their average, so as to conserver the trace of C
    
    lambdas_clean=lambdas
    lambdas_clean[lambdas_clean<=lambda_plus]=delta
    
    
    C_clean=np.zeros((N, N))
    v_m=np.matrix(v)
    
    for i in range(N-1):
        C_clean=C_clean+lambdas_clean[i] * np.dot(v_m[i,].T,v_m[i,]) 
        
    np.fill_diagonal(C_clean,1)
            
    return C_clean    


def solution_eig(C_asset):
    # C_corr = C_asset.corr()
    C_cov = C_asset.cov()
    l_e, V_e = LA.eig(C_cov)
    T, N = C_asset.shape
    q = N/T
    lambda_plus = (1+np.sqrt(q))**2
    
    C_clipped=eigenvalue_clipping(l_e,V_e,lambda_plus)
    return C_clipped

def weights_GVM(Sigma):
    Sigma_inv=LA.inv(Sigma)
    w_GVM=Sigma_inv.sum(axis=1)/Sigma_inv.sum()
    return w_GVM
    

## Load pre-processed & merged data, start computation

In [11]:
all_data = pd.read_pickle("data/clean/whole_usdt_merge.pkl")
all_data = all_data.reset_index()
all_data = all_data.drop(["time"], axis=1)

In [12]:
all_data

,1INCHDOWNUSDT,1INCHUPUSDT,1INCHUSDT,AAVEDOWNUSDT,AAVEUPUSDT,AAVEUSDT,ACAUSDT,ACHUSDT,ACMUSDT,ADADOWNUSDT,...,XVSUSDT,YFIDOWNUSDT,YFIIUSDT,YFIUPUSDT,YFIUSDT,YGGUSDT,ZECUSDT,ZENUSDT,ZILUSDT,ZRXUSDT
0,10.07,9.87,5.8799,1.327380,110.463,438.825,NaN,NaN,12.062,2.250567,...,85.699,0.001495,3044.99,8.890,50826.78,NaN,246.49,96.655,0.20656,2.1644
1,10.21,9.73,5.8589,1.367000,108.459,434.066,NaN,NaN,11.980,2.230800,...,84.578,0.001558,2989.17,8.530,50035.84,NaN,247.08,97.583,0.20509,2.1828
2,9.92,10.00,5.9137,1.320024,111.310,439.611,NaN,NaN,12.067,2.127842,...,83.166,0.001554,3000.41,8.550,50046.70,NaN,248.45,97.487,0.20619,2.1800
3,9.97,9.93,5.9084,1.301529,112.411,443.585,NaN,NaN,12.025,2.241671,...,81.401,0.001595,2954.89,8.301,49476.40,NaN,244.61,95.743,0.20330,2.1540
4,9.10,10.63,6.1123,1.232953,117.000,452.599,NaN,NaN,11.988,2.224815,...,82.750,0.001548,3000.85,8.642,50269.64,NaN,246.76,97.111,0.20582,2.1780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16447,NaN,NaN,NaN,NaN,NaN,80.100,0.1340,0.00926,2.947,0.003657,...,5.030,NaN,1355.00,NaN,7092.00,0.2532,45.30,10.480,0.02575,0.2025
16448,NaN,NaN,NaN,NaN,NaN,79.900,0.1324,0.00926,2.994,0.003627,...,5.100,NaN,1357.30,NaN,7106.00,0.2592,45.40,10.470,0.02584,0.2033
16449,NaN,NaN,NaN,NaN,NaN,79.700,0.1315,0.00920,3.115,0.003646,...,5.050,NaN,1357.10,NaN,7068.00,0.2581,45.30,10.480,0.02579,0.2037
16450,NaN,NaN,NaN,NaN,NaN,79.900,0.1312,0.00914,3.090,0.003607,...,5.050,NaN,1352.20,NaN,7086.00,0.2573,45.20,10.530,0.02595,0.2037


In [13]:
log_ret_all_data = np.log(all_data).diff()

In [14]:
log_ret_all_data

,1INCHDOWNUSDT,1INCHUPUSDT,1INCHUSDT,AAVEDOWNUSDT,AAVEUPUSDT,AAVEUSDT,ACAUSDT,ACHUSDT,ACMUSDT,ADADOWNUSDT,...,XVSUSDT,YFIDOWNUSDT,YFIIUSDT,YFIUPUSDT,YFIUSDT,YGGUSDT,ZECUSDT,ZENUSDT,ZILUSDT,ZRXUSDT
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.013807,-0.014286,-0.003578,0.029411,-0.018308,-0.010904,NaN,NaN,-0.006821,-0.008822,...,-0.013167,0.041277,-0.018502,-0.041338,-0.015684,NaN,0.002391,0.009555,-0.007142,0.008465
2,-0.028815,0.027371,0.009310,-0.034969,0.025947,0.012694,NaN,NaN,0.007236,-0.047252,...,-0.016836,-0.002571,0.003753,0.002342,0.000217,NaN,0.005529,-0.000984,0.005349,-0.001284
3,0.005028,-0.007025,-0.000897,-0.014110,0.009843,0.008999,NaN,NaN,-0.003487,0.052113,...,-0.021451,0.026041,-0.015288,-0.029555,-0.011461,NaN,-0.015577,-0.018052,-0.014115,-0.011998
4,-0.091306,0.068120,0.033928,-0.054128,0.040012,0.020117,NaN,NaN,-0.003082,-0.007548,...,0.016436,-0.029910,0.015434,0.040258,0.015906,NaN,0.008751,0.014187,0.012319,0.011080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16447,NaN,NaN,NaN,NaN,NaN,-0.014870,0.016554,-0.002157,0.041218,0.004934,...,0.001990,NaN,0.001182,NaN,0.008070,-0.010998,0.002210,-0.011385,-0.002715,-0.002466
16448,NaN,NaN,NaN,NaN,NaN,-0.002500,-0.012012,0.000000,0.015823,-0.008237,...,0.013821,NaN,0.001696,NaN,0.001972,0.023420,0.002205,-0.000955,0.003489,0.003943
16449,NaN,NaN,NaN,NaN,NaN,-0.002506,-0.006821,-0.006501,0.039619,0.005225,...,-0.009852,NaN,-0.000147,NaN,-0.005362,-0.004253,-0.002205,0.000955,-0.001937,0.001966
16450,NaN,NaN,NaN,NaN,NaN,0.002506,-0.002284,-0.006543,-0.008058,-0.010754,...,0.000000,NaN,-0.003617,NaN,0.002543,-0.003104,-0.002210,0.004760,0.006185,0.000000


In [15]:
t0 = 8000
t1 = 16000
X_raw = log_ret_all_data.iloc[t0:t1].dropna(axis=1)
X_raw.shape

(8000, 253)

In [16]:
X_raw

,AAVEUSDT,ACAUSDT,ACHUSDT,ACMUSDT,ADADOWNUSDT,ADAUPUSDT,ADAUSDT,ADXUSDT,AIONUSDT,AKROUSDT,...,XTZUSDT,XVGUSDT,XVSUSDT,YFIIUSDT,YFIUSDT,YGGUSDT,ZECUSDT,ZENUSDT,ZILUSDT,ZRXUSDT
8000,0.016944,0.037596,0.010463,0.008332,-0.015881,0.018140,0.009597,0.009128,0.010351,0.020501,...,0.010782,0.013205,0.011891,0.012456,0.015505,0.017925,0.013363,0.009248,0.014115,0.018657
8001,0.002685,0.008717,0.009066,-0.013926,-0.001256,0.002657,0.000000,0.000491,0.003427,0.012102,...,0.012323,-0.003032,-0.008309,0.010992,0.003121,0.012562,0.001106,0.002102,0.005104,0.004242
8002,-0.010782,-0.020038,-0.002582,-0.001531,0.004077,-0.007145,-0.002869,0.000736,-0.002283,0.000802,...,-0.010649,-0.005074,-0.002387,0.010439,-0.006650,-0.015984,-0.004430,-0.003155,-0.004214,-0.002765
8003,0.009440,0.023133,0.003613,0.001786,-0.008329,0.011086,0.003824,0.006844,0.002283,0.013530,...,0.012303,0.003047,0.005956,0.013752,0.004078,0.011588,0.012135,0.025219,0.006646,0.004604
8004,0.004019,-0.003095,0.005395,0.000000,-0.003794,0.002833,0.001907,0.003647,0.003415,0.014129,...,0.004287,0.005058,0.004739,0.033574,0.020826,0.004396,0.014154,0.020079,0.003086,0.009690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,0.005231,-0.000821,0.002488,0.004274,0.001206,0.004728,0.000384,0.000000,0.002977,-0.012012,...,0.003889,0.000000,0.000000,-0.001065,0.004000,0.004185,0.002581,0.005479,0.005821,0.010609
15996,-0.008734,-0.002468,-0.002488,-0.003417,0.007924,-0.014252,-0.004621,-0.001693,-0.005216,0.023882,...,-0.003889,-0.007092,-0.002621,-0.002545,-0.004729,-0.005384,-0.002581,0.002183,-0.005237,-0.003108
15997,-0.003515,-0.004127,-0.006246,-0.001713,0.004891,0.000000,-0.004255,0.000000,0.008555,-0.008889,...,-0.005208,0.007092,0.000000,0.000329,-0.002190,-0.010250,0.000000,-0.004372,-0.003507,0.001245
15998,-0.005296,-0.004144,0.013690,0.002996,0.004749,-0.014458,-0.003106,0.001693,-0.000741,0.005935,...,-0.006549,0.003527,0.005236,0.000821,-0.001280,-0.003643,-0.002587,-0.009907,-0.006461,0.000622


In [17]:
all_data_length = X_raw.shape[0]
T = 500
T_test = 2500
N = 253

T_in_list = [x*100 for x in range(3, 15)]

step_range = 4000
step_size = 5

In [18]:
@dask.delayed
def run_one_iteration(i, window, method="no"):
    # |train|    test     |
    # |--T--|------T------|
    C_asset_in = X_raw.iloc[i:i+window]
    C_asset_out = X_raw.iloc[i+window:i+window+T_test]

    # covariance matrix
    C_in_cov = C_asset_in.cov().values
    C_out_cov = C_asset_out.cov().values
    
    if method=="no":
        # 1. no correlation cleaning
        w = weights_GVM(C_in_cov)

    elif method=="eig":
        # 2. eigenvalue clipping
        C_clipped_in = solution_eig(C_asset_in)
        w = weights_GVM(C_clipped_in)

    elif method=="bahc":
        # 3. cleaning with bahc
        X_centered = sklearn.preprocessing.StandardScaler(with_mean=True,
                                with_std=False).fit_transform(C_asset_in.values)  # column-wise!'
        Sigma_BAHC=bahc.filterCovariance(X_centered.T)
        w=weights_GVM(Sigma_BAHC)

    elif method=="nls":
        # 4. cleaning with nls
        X_centered = sklearn.preprocessing.StandardScaler(with_mean=True,
                                with_std=False).fit_transform(C_asset_in.values)  # column-wise!'
        Sigma_NLS=pyRMT.optimalShrinkage(X_centered, return_covariance=True)
        w=weights_GVM(Sigma_NLS)

    # in sample risk
    sigma_in = w.T@(C_in_cov@w)
    
    # out sample risk
    sigma_out = w.T@(C_out_cov@w)


    return sigma_in, sigma_out

In [19]:
def run_one_T(t=300, method="no"):
    arr = {"in": [], "out": []}

    promises = [run_one_iteration(i, t, method) for i in range(0, step_range, step_size)]
    res = dask.compute(promises)

    for each in res[0]:
        sigma_in, sigma_out = each
        arr["in"].append(sigma_in)
        arr["out"].append(sigma_out)
        
    res_df = pd.DataFrame()
    res_df[f"{method}_arr_in"] = arr["in"]

    res_df[f"{method}_arr_out"] = arr["out"]

    res_df.to_pickle(f"data/clean/in_out_risk_{method}_{t}.pkl")

    print(res_df.mean())

### Uncomment the following block to run no/eig/nls/bahc methods on crypto data

In [20]:
# for each in ["no", "eig", "nls", "bahc"]:
#     for i in tqdm(range(len(T_in_list))):
#         run_one_T(T_in_list[i], each)